In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import math
import os
import time
import pandas as pd
from sklearn.model_selection import StratifiedKFold


LOG_DIR = '/content/drive/MyDrive/Clathrin-msCNN/log/esm2'
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
# Load dữ liệu
X_train = np.load('/content/drive/MyDrive/Clathrin-msCNN/data/t33_650M_UR50D/X_train.npy')
y_train = np.load('/content/drive/MyDrive/Clathrin-msCNN/data/t33_650M_UR50D/y_train.npy')
X_test = np.load('/content/drive/MyDrive/Clathrin-msCNN/data/t33_650M_UR50D/X_test.npy')
y_test = np.load('/content/drive/MyDrive/Clathrin-msCNN/data/t33_650M_UR50D/y_test.npy')


print("Kích thước dữ liệu:")
print(f"X_cv: {X_train.shape}")
print(f"y_cv: {y_train.shape}")
print(f"X_ind: {X_test.shape}")
print(f"y_ind: {y_test.shape}")

In [ ]:
X_train = X_train[:, np.newaxis, :, :]  # (samples, 1, height, width)
X_test = X_test[:, np.newaxis, :, :]

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (2421, 1, 1022, 1280)
X_test shape: (485, 1, 1022, 1280)


#Cross Validation

In [ ]:
import os
import time
import math

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.saving import register_keras_serializable

# === Hyperpararmeters ===
BATCH_SIZE      = 128
NUM_CLASSES     = 1
EPOCHS          = 50
NUM_FILTERS     = 256
NUM_HIDDEN      = 1024
WINDOW_SIZES    = [32]
MAX_SEQ_LENGTH  = 1022
EMBEDDING_WIDTH = 1280
LOG_MODEL = os.path.join(LOG_DIR, f'MODELS_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H')
os.makedirs(LOG_MODEL, exist_ok=True)

@register_keras_serializable()
def DeepScan(input_shape=(1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH),
             window_sizes=WINDOW_SIZES,
             num_filters=NUM_FILTERS,
             num_hidden=NUM_HIDDEN,
             num_classes=NUM_CLASSES):
    inputs = tf.keras.Input(shape=input_shape)

    branches = []
    for ws in window_sizes:
        x = layers.SeparableConv2D(
            filters=num_filters,
            kernel_size=(1, ws),
            strides=(1, 1),
            activation='relu',
            padding='valid',
            depthwise_regularizer=tf.keras.regularizers.l2(1e-4),
            pointwise_regularizer=tf.keras.regularizers.l2(1e-4),
            depthwise_initializer='glorot_uniform',
            pointwise_initializer='glorot_uniform'
        )(inputs)

        x = layers.MaxPooling2D(
            pool_size=(1, MAX_SEQ_LENGTH - ws + 1),
            strides=(1, 1),
            padding='valid'
        )(x)

        x = layers.Flatten()(x)
        branches.append(x)

    x = layers.Concatenate()(branches)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(num_hidden, activation='relu', name='fc1')(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='DeepScan')
    return model

# === Callback on CV folds ===
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val, fold):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.fold  = fold
        self.fold_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)
        cm = metrics.confusion_matrix(self.y_val, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()
        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0
        epoch_time = time.time() - self.epoch_start
        results.loc[len(results)] = [
            'CV', self.fold, epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, None,
            None, epoch_time, self.model.count_params()
        ]

class SaveEveryEpochCallback(tf.keras.callbacks.Callback):
    def __init__(self, base_dir, stage='CV', fold=None):
        super().__init__()
        self.base_dir = base_dir
        self.stage = stage
        self.fold = fold
        self.sub_dir = os.path.join(base_dir, stage, f'fold_{fold}' if fold is not None else '')
        os.makedirs(self.sub_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        filename = f"{self.stage}_fold{self.fold}_epoch{epoch+1:02d}.keras" if self.fold is not None else f"{self.stage}_epoch{epoch+1:02d}.keras"
        path = os.path.join(self.sub_dir, filename)
        self.model.save(path)
        print(f"[Saved model] Epoch {epoch+1} saved to {path}")

# === Callback on independent test ===
class FinalMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.epoch_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_test, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)

        cm = metrics.confusion_matrix(self.y_test, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()

        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0

        fpr, tpr, _ = roc_curve(self.y_test, y_pred_probs)
        roc_auc = auc(fpr, tpr)

        epoch_time = time.time() - self.epoch_start_time

        results.loc[len(results)] = [
            'Independent', 'Final', epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, roc_auc,
            None, epoch_time, self.model.count_params()
        ]

# === DataFrame Column ===
results_columns = [
    'Stage', 'Fold', 'Epoch', 'TP', 'FP', 'TN', 'FN',
    'Sens', 'Spec', 'Acc', 'MCC', 'F1', 'AUC',
    'Train_Time', 'Epoch_Time', 'Total_Params'
]
results = pd.DataFrame(columns=results_columns)

# === 5-Fold Cross-Validation + ROC per fold ===
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
    print(f"\n=== Fold {fold}/5 ===")
    X_train_fold, X_val = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val = y_train[train_idx], y_train[val_idx]

    model = DeepScan(window_sizes=WINDOW_SIZES,
                     num_filters=NUM_FILTERS,
                     num_hidden=NUM_HIDDEN)
    model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
    model.summary()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    metrics_cb = MetricsCallback(X_val, y_val, fold)
    tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, f'fold_{fold}'))
    early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='CV', fold=fold)
    checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_CV_{fold}.keras')
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')

    start_fold = time.time()
    model.fit(
        X_train_fold, y_train_fold,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=[metrics_cb, tb_cb, early_stop_cb, save_cb, checkpoint_cb],
        verbose=1,
    )
    train_time = time.time() - start_fold
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'Train_Time'] = train_time

    y_val_probs = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
    fpr, tpr, _ = roc_curve(y_val, y_val_probs)
    roc_auc = auc(fpr, tpr)
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'AUC'] = roc_auc

# Save CV results
results_path = os.path.join(
    LOG_DIR,
    f'training_results_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H.csv'
)
results.to_csv(results_path, index=False)
print(f"\nCV results saved to {results_path}")

# === Final training and evaluation ===
final_cb = FinalMetricsCallback(X_test, y_test)

final_model = DeepScan(window_sizes=WINDOW_SIZES,
                      num_filters=NUM_FILTERS,
                      num_hidden=NUM_HIDDEN)
final_model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='Independent')
final_checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_Independent.keras')
final_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(final_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')
final_early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
final_tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, 'final_model'))

history = final_model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[final_cb, final_tb_cb, final_checkpoint_cb, final_early_stop_cb, save_cb],
    verbose=1
)

# Save final results
results.to_csv(results_path, index=False)
print(f"\nFinal results saved to {results_path}")

best_auc_idx = results[results['Stage'] == 'Independent']['AUC'].idxmax()
best_auc_row = results.loc[best_auc_idx]
print(f"\nBest Independent Test AUC: {best_auc_row['AUC']:.4f} (Epoch {best_auc_row['Epoch']})")



=== Fold 1/5 ===


Model: "DeepScan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d                │ (None, 1, 991, 256)    │       368,896 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate (Concatenate)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 633,089 (2.42 MB)

 Trainable params: 633,089 (2.42 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_4              │ (None, 1, 991, 256)    │       368,896 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_4 (Concatenate)     │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 633,089 (2.42 MB)

 Trainable params: 633,089 (2.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step - accuracy: 0.5430 - auc: 0.5488 - loss: 0.7478[Saved model] Epoch 1 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[32]_256F_1024H/CV/fold_5/CV_fold5_epoch01.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 12s 702ms/step - accuracy: 0.5471 - auc: 0.5559 - loss: 0.7443 - val_accuracy: 0.6880 - val_auc: 0.9034 - val_loss: 0.6273
Epoch 2/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.7039 - auc: 0.7735 - loss: 0.6202[Saved model] Epoch 2 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[32]_256F_1024H/CV/fold_5/CV_fold5_epoch02.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 540ms/step - accuracy: 0.7066 - auc: 0.7767 - loss: 0.6177 - val_accuracy: 0.6715 - val_auc: 0.9262 - val_loss: 0.5798
Epoch 3/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step - accuracy: 0.7427 - auc: 0.8552 - loss: 0.5461[Saved model] Epoch 3 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[32]_256F_1024H/CV/fold_5/CV_fold5_epoch03.keras
16

In [ ]:
import os
import time
import math

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.saving import register_keras_serializable

# === Hyperpararmeters ===
BATCH_SIZE      = 128
NUM_CLASSES     = 1
EPOCHS          = 50
NUM_FILTERS     = 256
NUM_HIDDEN      = 1024
WINDOW_SIZES    = [28]
MAX_SEQ_LENGTH  = 1022
EMBEDDING_WIDTH = 1280
LOG_MODEL = os.path.join(LOG_DIR, f'MODELS_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H')
os.makedirs(LOG_MODEL, exist_ok=True)

@register_keras_serializable()
def DeepScan(input_shape=(1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH),
             window_sizes=WINDOW_SIZES,
             num_filters=NUM_FILTERS,
             num_hidden=NUM_HIDDEN,
             num_classes=NUM_CLASSES):
    inputs = tf.keras.Input(shape=input_shape)

    branches = []
    for ws in window_sizes:
        x = layers.SeparableConv2D(
            filters=num_filters,
            kernel_size=(1, ws),
            strides=(1, 1),
            activation='relu',
            padding='valid',
            depthwise_regularizer=tf.keras.regularizers.l2(1e-4),
            pointwise_regularizer=tf.keras.regularizers.l2(1e-4),
            depthwise_initializer='glorot_uniform',
            pointwise_initializer='glorot_uniform'
        )(inputs)

        x = layers.MaxPooling2D(
            pool_size=(1, MAX_SEQ_LENGTH - ws + 1),
            strides=(1, 1),
            padding='valid'
        )(x)

        x = layers.Flatten()(x)
        branches.append(x)

    x = layers.Concatenate()(branches)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(num_hidden, activation='relu', name='fc1')(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='DeepScan')
    return model

# === Callback on CV folds ===
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val, fold):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.fold  = fold
        self.fold_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)
        cm = metrics.confusion_matrix(self.y_val, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()
        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0
        epoch_time = time.time() - self.epoch_start
        results.loc[len(results)] = [
            'CV', self.fold, epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, None,
            None, epoch_time, self.model.count_params()
        ]

class SaveEveryEpochCallback(tf.keras.callbacks.Callback):
    def __init__(self, base_dir, stage='CV', fold=None):
        super().__init__()
        self.base_dir = base_dir
        self.stage = stage
        self.fold = fold
        self.sub_dir = os.path.join(base_dir, stage, f'fold_{fold}' if fold is not None else '')
        os.makedirs(self.sub_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        filename = f"{self.stage}_fold{self.fold}_epoch{epoch+1:02d}.keras" if self.fold is not None else f"{self.stage}_epoch{epoch+1:02d}.keras"
        path = os.path.join(self.sub_dir, filename)
        self.model.save(path)
        print(f"[Saved model] Epoch {epoch+1} saved to {path}")

# === Callback on independent test ===
class FinalMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.epoch_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_test, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)

        cm = metrics.confusion_matrix(self.y_test, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()

        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0

        fpr, tpr, _ = roc_curve(self.y_test, y_pred_probs)
        roc_auc = auc(fpr, tpr)

        epoch_time = time.time() - self.epoch_start_time

        results.loc[len(results)] = [
            'Independent', 'Final', epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, roc_auc,
            None, epoch_time, self.model.count_params()
        ]

# === DataFrame Column ===
results_columns = [
    'Stage', 'Fold', 'Epoch', 'TP', 'FP', 'TN', 'FN',
    'Sens', 'Spec', 'Acc', 'MCC', 'F1', 'AUC',
    'Train_Time', 'Epoch_Time', 'Total_Params'
]
results = pd.DataFrame(columns=results_columns)

# === 5-Fold Cross-Validation + ROC per fold ===
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
    print(f"\n=== Fold {fold}/5 ===")
    X_train_fold, X_val = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val = y_train[train_idx], y_train[val_idx]

    model = DeepScan(window_sizes=WINDOW_SIZES,
                     num_filters=NUM_FILTERS,
                     num_hidden=NUM_HIDDEN)
    model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
    model.summary()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    metrics_cb = MetricsCallback(X_val, y_val, fold)
    tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, f'fold_{fold}'))
    early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='CV', fold=fold)
    checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_CV_{fold}.keras')
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')

    start_fold = time.time()
    model.fit(
        X_train_fold, y_train_fold,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=[metrics_cb, tb_cb, early_stop_cb, save_cb, checkpoint_cb],
        verbose=1,
    )
    train_time = time.time() - start_fold
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'Train_Time'] = train_time

    y_val_probs = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
    fpr, tpr, _ = roc_curve(y_val, y_val_probs)
    roc_auc = auc(fpr, tpr)
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'AUC'] = roc_auc

# Save CV results
results_path = os.path.join(
    LOG_DIR,
    f'training_results_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H.csv'
)
results.to_csv(results_path, index=False)
print(f"\nCV results saved to {results_path}")

# === Final training and evaluation ===
final_cb = FinalMetricsCallback(X_test, y_test)

final_model = DeepScan(window_sizes=WINDOW_SIZES,
                      num_filters=NUM_FILTERS,
                      num_hidden=NUM_HIDDEN)
final_model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='Independent')
final_checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_Independent.keras')
final_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(final_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')
final_early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
final_tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, 'final_model'))

history = final_model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[final_cb, final_tb_cb, final_checkpoint_cb, final_early_stop_cb, save_cb],
    verbose=1
)

# Save final results
results.to_csv(results_path, index=False)
print(f"\nFinal results saved to {results_path}")

best_auc_idx = results[results['Stage'] == 'Independent']['AUC'].idxmax()
best_auc_row = results.loc[best_auc_idx]
print(f"\nBest Independent Test AUC: {best_auc_row['AUC']:.4f} (Epoch {best_auc_row['Epoch']})")



=== Fold 1/5 ===


Model: "DeepScan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_6              │ (None, 1, 995, 256)    │       363,776 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_6 (Concatenate)     │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 627,969 (2.40 MB)

 Trainable params: 627,969 (2.40 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_10             │ (None, 1, 995, 256)    │       363,776 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_10 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 627,969 (2.40 MB)

 Trainable params: 627,969 (2.40 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.5348 - auc: 0.5511 - loss: 0.8721[Saved model] Epoch 1 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[28]_256F_1024H/CV/fold_5/CV_fold5_epoch01.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 676ms/step - accuracy: 0.5384 - auc: 0.5544 - loss: 0.8624 - val_accuracy: 0.7686 - val_auc: 0.8619 - val_loss: 0.6485
Epoch 2/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - accuracy: 0.6471 - auc: 0.7363 - loss: 0.6636[Saved model] Epoch 2 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[28]_256F_1024H/CV/fold_5/CV_fold5_epoch02.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 521ms/step - accuracy: 0.6505 - auc: 0.7374 - loss: 0.6607 - val_accuracy: 0.6736 - val_auc: 0.9223 - val_loss: 0.5940
Epoch 3/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step - accuracy: 0.7525 - auc: 0.8415 - loss: 0.5627[Saved model] Epoch 3 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[28]_256F_1024H/CV/fold_5/CV_fold5_epoch03.keras
16

In [ ]:
import os
import time
import math

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.saving import register_keras_serializable

# === Hyperpararmeters ===
BATCH_SIZE      = 128
NUM_CLASSES     = 1
EPOCHS          = 50
NUM_FILTERS     = 256
NUM_HIDDEN      = 1024
WINDOW_SIZES    = [24]
MAX_SEQ_LENGTH  = 1022
EMBEDDING_WIDTH = 1280
LOG_MODEL = os.path.join(LOG_DIR, f'MODELS_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H')
os.makedirs(LOG_MODEL, exist_ok=True)

@register_keras_serializable()
def DeepScan(input_shape=(1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH),
             window_sizes=WINDOW_SIZES,
             num_filters=NUM_FILTERS,
             num_hidden=NUM_HIDDEN,
             num_classes=NUM_CLASSES):
    inputs = tf.keras.Input(shape=input_shape)

    branches = []
    for ws in window_sizes:
        x = layers.SeparableConv2D(
            filters=num_filters,
            kernel_size=(1, ws),
            strides=(1, 1),
            activation='relu',
            padding='valid',
            depthwise_regularizer=tf.keras.regularizers.l2(1e-4),
            pointwise_regularizer=tf.keras.regularizers.l2(1e-4),
            depthwise_initializer='glorot_uniform',
            pointwise_initializer='glorot_uniform'
        )(inputs)

        x = layers.MaxPooling2D(
            pool_size=(1, MAX_SEQ_LENGTH - ws + 1),
            strides=(1, 1),
            padding='valid'
        )(x)

        x = layers.Flatten()(x)
        branches.append(x)

    x = layers.Concatenate()(branches)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(num_hidden, activation='relu', name='fc1')(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='DeepScan')
    return model

# === Callback on CV folds ===
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val, fold):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.fold  = fold
        self.fold_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)
        cm = metrics.confusion_matrix(self.y_val, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()
        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0
        epoch_time = time.time() - self.epoch_start
        results.loc[len(results)] = [
            'CV', self.fold, epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, None,
            None, epoch_time, self.model.count_params()
        ]

class SaveEveryEpochCallback(tf.keras.callbacks.Callback):
    def __init__(self, base_dir, stage='CV', fold=None):
        super().__init__()
        self.base_dir = base_dir
        self.stage = stage
        self.fold = fold
        self.sub_dir = os.path.join(base_dir, stage, f'fold_{fold}' if fold is not None else '')
        os.makedirs(self.sub_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        filename = f"{self.stage}_fold{self.fold}_epoch{epoch+1:02d}.keras" if self.fold is not None else f"{self.stage}_epoch{epoch+1:02d}.keras"
        path = os.path.join(self.sub_dir, filename)
        self.model.save(path)
        print(f"[Saved model] Epoch {epoch+1} saved to {path}")

# === Callback on independent test ===
class FinalMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.epoch_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_test, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)

        cm = metrics.confusion_matrix(self.y_test, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()

        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0

        fpr, tpr, _ = roc_curve(self.y_test, y_pred_probs)
        roc_auc = auc(fpr, tpr)

        epoch_time = time.time() - self.epoch_start_time

        results.loc[len(results)] = [
            'Independent', 'Final', epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, roc_auc,
            None, epoch_time, self.model.count_params()
        ]

# === DataFrame Column ===
results_columns = [
    'Stage', 'Fold', 'Epoch', 'TP', 'FP', 'TN', 'FN',
    'Sens', 'Spec', 'Acc', 'MCC', 'F1', 'AUC',
    'Train_Time', 'Epoch_Time', 'Total_Params'
]
results = pd.DataFrame(columns=results_columns)

# === 5-Fold Cross-Validation + ROC per fold ===
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
    print(f"\n=== Fold {fold}/5 ===")
    X_train_fold, X_val = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val = y_train[train_idx], y_train[val_idx]

    model = DeepScan(window_sizes=WINDOW_SIZES,
                     num_filters=NUM_FILTERS,
                     num_hidden=NUM_HIDDEN)
    model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
    model.summary()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    metrics_cb = MetricsCallback(X_val, y_val, fold)
    tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, f'fold_{fold}'))
    early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='CV', fold=fold)
    checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_CV_{fold}.keras')
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')

    start_fold = time.time()
    model.fit(
        X_train_fold, y_train_fold,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=[metrics_cb, tb_cb, early_stop_cb, save_cb, checkpoint_cb],
        verbose=1,
    )
    train_time = time.time() - start_fold
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'Train_Time'] = train_time

    y_val_probs = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
    fpr, tpr, _ = roc_curve(y_val, y_val_probs)
    roc_auc = auc(fpr, tpr)
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'AUC'] = roc_auc

# Save CV results
results_path = os.path.join(
    LOG_DIR,
    f'training_results_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H.csv'
)
results.to_csv(results_path, index=False)
print(f"\nCV results saved to {results_path}")

# === Final training and evaluation ===
final_cb = FinalMetricsCallback(X_test, y_test)

final_model = DeepScan(window_sizes=WINDOW_SIZES,
                      num_filters=NUM_FILTERS,
                      num_hidden=NUM_HIDDEN)
final_model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='Independent')
final_checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_Independent.keras')
final_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(final_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')
final_early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
final_tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, 'final_model'))

history = final_model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[final_cb, final_tb_cb, final_checkpoint_cb, final_early_stop_cb, save_cb],
    verbose=1
)

# Save final results
results.to_csv(results_path, index=False)
print(f"\nFinal results saved to {results_path}")

best_auc_idx = results[results['Stage'] == 'Independent']['AUC'].idxmax()
best_auc_row = results.loc[best_auc_idx]
print(f"\nBest Independent Test AUC: {best_auc_row['AUC']:.4f} (Epoch {best_auc_row['Epoch']})")



=== Fold 1/5 ===


Model: "DeepScan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_12             │ (None, 1, 999, 256)    │       358,656 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_12 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 622,849 (2.38 MB)

 Trainable params: 622,849 (2.38 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_16             │ (None, 1, 999, 256)    │       358,656 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_16 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 622,849 (2.38 MB)

 Trainable params: 622,849 (2.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step - accuracy: 0.5499 - auc: 0.5634 - loss: 0.7392[Saved model] Epoch 1 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[24]_256F_1024H/CV/fold_5/CV_fold5_epoch01.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 678ms/step - accuracy: 0.5519 - auc: 0.5674 - loss: 0.7376 - val_accuracy: 0.5640 - val_auc: 0.8680 - val_loss: 0.6736
Epoch 2/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step - accuracy: 0.6582 - auc: 0.7343 - loss: 0.6513[Saved model] Epoch 2 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[24]_256F_1024H/CV/fold_5/CV_fold5_epoch02.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 522ms/step - accuracy: 0.6610 - auc: 0.7364 - loss: 0.6492 - val_accuracy: 0.7872 - val_auc: 0.9147 - val_loss: 0.5874
Epoch 3/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.7247 - auc: 0.8255 - loss: 0.5759[Saved model] Epoch 3 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[24]_256F_1024H/CV/fold_5/CV_fold5_epoch03.keras
16

In [ ]:
import os
import time
import math

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.saving import register_keras_serializable

# === Hyperpararmeters ===
BATCH_SIZE      = 128
NUM_CLASSES     = 1
EPOCHS          = 50
NUM_FILTERS     = 256
NUM_HIDDEN      = 1024
WINDOW_SIZES    = [20]
MAX_SEQ_LENGTH  = 1022
EMBEDDING_WIDTH = 1280
LOG_MODEL = os.path.join(LOG_DIR, f'MODELS_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H')
os.makedirs(LOG_MODEL, exist_ok=True)

@register_keras_serializable()
def DeepScan(input_shape=(1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH),
             window_sizes=WINDOW_SIZES,
             num_filters=NUM_FILTERS,
             num_hidden=NUM_HIDDEN,
             num_classes=NUM_CLASSES):
    inputs = tf.keras.Input(shape=input_shape)

    branches = []
    for ws in window_sizes:
        x = layers.SeparableConv2D(
            filters=num_filters,
            kernel_size=(1, ws),
            strides=(1, 1),
            activation='relu',
            padding='valid',
            depthwise_regularizer=tf.keras.regularizers.l2(1e-4),
            pointwise_regularizer=tf.keras.regularizers.l2(1e-4),
            depthwise_initializer='glorot_uniform',
            pointwise_initializer='glorot_uniform'
        )(inputs)

        x = layers.MaxPooling2D(
            pool_size=(1, MAX_SEQ_LENGTH - ws + 1),
            strides=(1, 1),
            padding='valid'
        )(x)

        x = layers.Flatten()(x)
        branches.append(x)

    x = layers.Concatenate()(branches)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(num_hidden, activation='relu', name='fc1')(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='DeepScan')
    return model

# === Callback on CV folds ===
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val, fold):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.fold  = fold
        self.fold_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)
        cm = metrics.confusion_matrix(self.y_val, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()
        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0
        epoch_time = time.time() - self.epoch_start
        results.loc[len(results)] = [
            'CV', self.fold, epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, None,
            None, epoch_time, self.model.count_params()
        ]

class SaveEveryEpochCallback(tf.keras.callbacks.Callback):
    def __init__(self, base_dir, stage='CV', fold=None):
        super().__init__()
        self.base_dir = base_dir
        self.stage = stage
        self.fold = fold
        self.sub_dir = os.path.join(base_dir, stage, f'fold_{fold}' if fold is not None else '')
        os.makedirs(self.sub_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        filename = f"{self.stage}_fold{self.fold}_epoch{epoch+1:02d}.keras" if self.fold is not None else f"{self.stage}_epoch{epoch+1:02d}.keras"
        path = os.path.join(self.sub_dir, filename)
        self.model.save(path)
        print(f"[Saved model] Epoch {epoch+1} saved to {path}")

# === Callback on independent test ===
class FinalMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.epoch_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_test, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)

        cm = metrics.confusion_matrix(self.y_test, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()

        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0

        fpr, tpr, _ = roc_curve(self.y_test, y_pred_probs)
        roc_auc = auc(fpr, tpr)

        epoch_time = time.time() - self.epoch_start_time

        results.loc[len(results)] = [
            'Independent', 'Final', epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, roc_auc,
            None, epoch_time, self.model.count_params()
        ]

# === DataFrame Column ===
results_columns = [
    'Stage', 'Fold', 'Epoch', 'TP', 'FP', 'TN', 'FN',
    'Sens', 'Spec', 'Acc', 'MCC', 'F1', 'AUC',
    'Train_Time', 'Epoch_Time', 'Total_Params'
]
results = pd.DataFrame(columns=results_columns)

# === 5-Fold Cross-Validation + ROC per fold ===
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
    print(f"\n=== Fold {fold}/5 ===")
    X_train_fold, X_val = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val = y_train[train_idx], y_train[val_idx]

    model = DeepScan(window_sizes=WINDOW_SIZES,
                     num_filters=NUM_FILTERS,
                     num_hidden=NUM_HIDDEN)
    model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
    model.summary()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    metrics_cb = MetricsCallback(X_val, y_val, fold)
    tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, f'fold_{fold}'))
    early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='CV', fold=fold)
    checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_CV_{fold}.keras')
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')

    start_fold = time.time()
    model.fit(
        X_train_fold, y_train_fold,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=[metrics_cb, tb_cb, early_stop_cb, save_cb, checkpoint_cb],
        verbose=1,
    )
    train_time = time.time() - start_fold
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'Train_Time'] = train_time

    y_val_probs = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
    fpr, tpr, _ = roc_curve(y_val, y_val_probs)
    roc_auc = auc(fpr, tpr)
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'AUC'] = roc_auc

# Save CV results
results_path = os.path.join(
    LOG_DIR,
    f'training_results_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H.csv'
)
results.to_csv(results_path, index=False)
print(f"\nCV results saved to {results_path}")

# === Final training and evaluation ===
final_cb = FinalMetricsCallback(X_test, y_test)

final_model = DeepScan(window_sizes=WINDOW_SIZES,
                      num_filters=NUM_FILTERS,
                      num_hidden=NUM_HIDDEN)
final_model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='Independent')
final_checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_Independent.keras')
final_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(final_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')
final_early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
final_tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, 'final_model'))

history = final_model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[final_cb, final_tb_cb, final_checkpoint_cb, final_early_stop_cb, save_cb],
    verbose=1
)

# Save final results
results.to_csv(results_path, index=False)
print(f"\nFinal results saved to {results_path}")

best_auc_idx = results[results['Stage'] == 'Independent']['AUC'].idxmax()
best_auc_row = results.loc[best_auc_idx]
print(f"\nBest Independent Test AUC: {best_auc_row['AUC']:.4f} (Epoch {best_auc_row['Epoch']})")



=== Fold 1/5 ===


Model: "DeepScan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_18             │ (None, 1, 1003, 256)   │       353,536 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_18 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 617,729 (2.36 MB)

 Trainable params: 617,729 (2.36 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_22 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_22             │ (None, 1, 1003, 256)   │       353,536 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_22 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 617,729 (2.36 MB)

 Trainable params: 617,729 (2.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step - accuracy: 0.5374 - auc: 0.5292 - loss: 0.8001[Saved model] Epoch 1 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[20]_256F_1024H/CV/fold_5/CV_fold5_epoch01.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 677ms/step - accuracy: 0.5395 - auc: 0.5344 - loss: 0.7944 - val_accuracy: 0.6178 - val_auc: 0.8644 - val_loss: 0.6612
Epoch 2/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.6527 - auc: 0.7109 - loss: 0.6645[Saved model] Epoch 2 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[20]_256F_1024H/CV/fold_5/CV_fold5_epoch02.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 530ms/step - accuracy: 0.6557 - auc: 0.7151 - loss: 0.6621 - val_accuracy: 0.8120 - val_auc: 0.8986 - val_loss: 0.5954
Epoch 3/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step - accuracy: 0.7160 - auc: 0.8054 - loss: 0.5928[Saved model] Epoch 3 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[20]_256F_1024H/CV/fold_5/CV_fold5_epoch03.keras
16

In [ ]:
import os
import time
import math

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.saving import register_keras_serializable

# === Hyperpararmeters ===
BATCH_SIZE      = 128
NUM_CLASSES     = 1
EPOCHS          = 50
NUM_FILTERS     = 256
NUM_HIDDEN      = 1024
WINDOW_SIZES    = [16]
MAX_SEQ_LENGTH  = 1022
EMBEDDING_WIDTH = 1280
LOG_MODEL = os.path.join(LOG_DIR, f'MODELS_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H')
os.makedirs(LOG_MODEL, exist_ok=True)

@register_keras_serializable()
def DeepScan(input_shape=(1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH),
             window_sizes=WINDOW_SIZES,
             num_filters=NUM_FILTERS,
             num_hidden=NUM_HIDDEN,
             num_classes=NUM_CLASSES):
    inputs = tf.keras.Input(shape=input_shape)

    branches = []
    for ws in window_sizes:
        x = layers.SeparableConv2D(
            filters=num_filters,
            kernel_size=(1, ws),
            strides=(1, 1),
            activation='relu',
            padding='valid',
            depthwise_regularizer=tf.keras.regularizers.l2(1e-4),
            pointwise_regularizer=tf.keras.regularizers.l2(1e-4),
            depthwise_initializer='glorot_uniform',
            pointwise_initializer='glorot_uniform'
        )(inputs)

        x = layers.MaxPooling2D(
            pool_size=(1, MAX_SEQ_LENGTH - ws + 1),
            strides=(1, 1),
            padding='valid'
        )(x)

        x = layers.Flatten()(x)
        branches.append(x)

    x = layers.Concatenate()(branches)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(num_hidden, activation='relu', name='fc1')(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='DeepScan')
    return model

# === Callback on CV folds ===
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val, fold):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.fold  = fold
        self.fold_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)
        cm = metrics.confusion_matrix(self.y_val, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()
        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0
        epoch_time = time.time() - self.epoch_start
        results.loc[len(results)] = [
            'CV', self.fold, epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, None,
            None, epoch_time, self.model.count_params()
        ]

class SaveEveryEpochCallback(tf.keras.callbacks.Callback):
    def __init__(self, base_dir, stage='CV', fold=None):
        super().__init__()
        self.base_dir = base_dir
        self.stage = stage
        self.fold = fold
        self.sub_dir = os.path.join(base_dir, stage, f'fold_{fold}' if fold is not None else '')
        os.makedirs(self.sub_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        filename = f"{self.stage}_fold{self.fold}_epoch{epoch+1:02d}.keras" if self.fold is not None else f"{self.stage}_epoch{epoch+1:02d}.keras"
        path = os.path.join(self.sub_dir, filename)
        self.model.save(path)
        print(f"[Saved model] Epoch {epoch+1} saved to {path}")

# === Callback on independent test ===
class FinalMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.epoch_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_test, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)

        cm = metrics.confusion_matrix(self.y_test, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()

        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0

        fpr, tpr, _ = roc_curve(self.y_test, y_pred_probs)
        roc_auc = auc(fpr, tpr)

        epoch_time = time.time() - self.epoch_start_time

        results.loc[len(results)] = [
            'Independent', 'Final', epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, roc_auc,
            None, epoch_time, self.model.count_params()
        ]

# === DataFrame Column ===
results_columns = [
    'Stage', 'Fold', 'Epoch', 'TP', 'FP', 'TN', 'FN',
    'Sens', 'Spec', 'Acc', 'MCC', 'F1', 'AUC',
    'Train_Time', 'Epoch_Time', 'Total_Params'
]
results = pd.DataFrame(columns=results_columns)

# === 5-Fold Cross-Validation + ROC per fold ===
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
    print(f"\n=== Fold {fold}/5 ===")
    X_train_fold, X_val = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val = y_train[train_idx], y_train[val_idx]

    model = DeepScan(window_sizes=WINDOW_SIZES,
                     num_filters=NUM_FILTERS,
                     num_hidden=NUM_HIDDEN)
    model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
    model.summary()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    metrics_cb = MetricsCallback(X_val, y_val, fold)
    tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, f'fold_{fold}'))
    early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='CV', fold=fold)
    checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_CV_{fold}.keras')
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')

    start_fold = time.time()
    model.fit(
        X_train_fold, y_train_fold,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=[metrics_cb, tb_cb, early_stop_cb, save_cb, checkpoint_cb],
        verbose=1,
    )
    train_time = time.time() - start_fold
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'Train_Time'] = train_time

    y_val_probs = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
    fpr, tpr, _ = roc_curve(y_val, y_val_probs)
    roc_auc = auc(fpr, tpr)
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'AUC'] = roc_auc

# Save CV results
results_path = os.path.join(
    LOG_DIR,
    f'training_results_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H.csv'
)
results.to_csv(results_path, index=False)
print(f"\nCV results saved to {results_path}")

# === Final training and evaluation ===
final_cb = FinalMetricsCallback(X_test, y_test)

final_model = DeepScan(window_sizes=WINDOW_SIZES,
                      num_filters=NUM_FILTERS,
                      num_hidden=NUM_HIDDEN)
final_model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='Independent')
final_checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_Independent.keras')
final_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(final_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')
final_early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
final_tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, 'final_model'))

history = final_model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[final_cb, final_tb_cb, final_checkpoint_cb, final_early_stop_cb, save_cb],
    verbose=1
)

# Save final results
results.to_csv(results_path, index=False)
print(f"\nFinal results saved to {results_path}")

best_auc_idx = results[results['Stage'] == 'Independent']['AUC'].idxmax()
best_auc_row = results.loc[best_auc_idx]
print(f"\nBest Independent Test AUC: {best_auc_row['AUC']:.4f} (Epoch {best_auc_row['Epoch']})")



=== Fold 1/5 ===


Model: "DeepScan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_24 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_24             │ (None, 1, 1007, 256)   │       348,416 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_24 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_24 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 612,609 (2.34 MB)

 Trainable params: 612,609 (2.34 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_28 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_28             │ (None, 1, 1007, 256)   │       348,416 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_28 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_28 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 612,609 (2.34 MB)

 Trainable params: 612,609 (2.34 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.5200 - auc: 0.5351 - loss: 0.8052[Saved model] Epoch 1 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[16]_256F_1024H/CV/fold_5/CV_fold5_epoch01.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 676ms/step - accuracy: 0.5238 - auc: 0.5382 - loss: 0.8008 - val_accuracy: 0.5971 - val_auc: 0.8742 - val_loss: 0.6772
Epoch 2/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - accuracy: 0.6424 - auc: 0.7010 - loss: 0.6723[Saved model] Epoch 2 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[16]_256F_1024H/CV/fold_5/CV_fold5_epoch02.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 525ms/step - accuracy: 0.6421 - auc: 0.7022 - loss: 0.6712 - val_accuracy: 0.7624 - val_auc: 0.9136 - val_loss: 0.6129
Epoch 3/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step - accuracy: 0.7155 - auc: 0.7900 - loss: 0.6092[Saved model] Epoch 3 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[16]_256F_1024H/CV/fold_5/CV_fold5_epoch03.keras
16

In [ ]:
import os
import time
import math

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.saving import register_keras_serializable

# === Hyperpararmeters ===
BATCH_SIZE      = 128
NUM_CLASSES     = 1
EPOCHS          = 50
NUM_FILTERS     = 256
NUM_HIDDEN      = 1024
WINDOW_SIZES    = [12]
MAX_SEQ_LENGTH  = 1022
EMBEDDING_WIDTH = 1280
LOG_MODEL = os.path.join(LOG_DIR, f'MODELS_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H')
os.makedirs(LOG_MODEL, exist_ok=True)

@register_keras_serializable()
def DeepScan(input_shape=(1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH),
             window_sizes=WINDOW_SIZES,
             num_filters=NUM_FILTERS,
             num_hidden=NUM_HIDDEN,
             num_classes=NUM_CLASSES):
    inputs = tf.keras.Input(shape=input_shape)

    branches = []
    for ws in window_sizes:
        x = layers.SeparableConv2D(
            filters=num_filters,
            kernel_size=(1, ws),
            strides=(1, 1),
            activation='relu',
            padding='valid',
            depthwise_regularizer=tf.keras.regularizers.l2(1e-4),
            pointwise_regularizer=tf.keras.regularizers.l2(1e-4),
            depthwise_initializer='glorot_uniform',
            pointwise_initializer='glorot_uniform'
        )(inputs)

        x = layers.MaxPooling2D(
            pool_size=(1, MAX_SEQ_LENGTH - ws + 1),
            strides=(1, 1),
            padding='valid'
        )(x)

        x = layers.Flatten()(x)
        branches.append(x)

    x = layers.Concatenate()(branches)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(num_hidden, activation='relu', name='fc1')(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='DeepScan')
    return model

# === Callback on CV folds ===
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val, fold):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.fold  = fold
        self.fold_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)
        cm = metrics.confusion_matrix(self.y_val, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()
        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0
        epoch_time = time.time() - self.epoch_start
        results.loc[len(results)] = [
            'CV', self.fold, epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, None,
            None, epoch_time, self.model.count_params()
        ]

class SaveEveryEpochCallback(tf.keras.callbacks.Callback):
    def __init__(self, base_dir, stage='CV', fold=None):
        super().__init__()
        self.base_dir = base_dir
        self.stage = stage
        self.fold = fold
        self.sub_dir = os.path.join(base_dir, stage, f'fold_{fold}' if fold is not None else '')
        os.makedirs(self.sub_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        filename = f"{self.stage}_fold{self.fold}_epoch{epoch+1:02d}.keras" if self.fold is not None else f"{self.stage}_epoch{epoch+1:02d}.keras"
        path = os.path.join(self.sub_dir, filename)
        self.model.save(path)
        print(f"[Saved model] Epoch {epoch+1} saved to {path}")

# === Callback on independent test ===
class FinalMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.epoch_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_test, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)

        cm = metrics.confusion_matrix(self.y_test, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()

        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0

        fpr, tpr, _ = roc_curve(self.y_test, y_pred_probs)
        roc_auc = auc(fpr, tpr)

        epoch_time = time.time() - self.epoch_start_time

        results.loc[len(results)] = [
            'Independent', 'Final', epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, roc_auc,
            None, epoch_time, self.model.count_params()
        ]

# === DataFrame Column ===
results_columns = [
    'Stage', 'Fold', 'Epoch', 'TP', 'FP', 'TN', 'FN',
    'Sens', 'Spec', 'Acc', 'MCC', 'F1', 'AUC',
    'Train_Time', 'Epoch_Time', 'Total_Params'
]
results = pd.DataFrame(columns=results_columns)

# === 5-Fold Cross-Validation + ROC per fold ===
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
    print(f"\n=== Fold {fold}/5 ===")
    X_train_fold, X_val = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val = y_train[train_idx], y_train[val_idx]

    model = DeepScan(window_sizes=WINDOW_SIZES,
                     num_filters=NUM_FILTERS,
                     num_hidden=NUM_HIDDEN)
    model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
    model.summary()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    metrics_cb = MetricsCallback(X_val, y_val, fold)
    tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, f'fold_{fold}'))
    early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='CV', fold=fold)
    checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_CV_{fold}.keras')
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')

    start_fold = time.time()
    model.fit(
        X_train_fold, y_train_fold,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=[metrics_cb, tb_cb, early_stop_cb, save_cb, checkpoint_cb],
        verbose=1,
    )
    train_time = time.time() - start_fold
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'Train_Time'] = train_time

    y_val_probs = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
    fpr, tpr, _ = roc_curve(y_val, y_val_probs)
    roc_auc = auc(fpr, tpr)
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'AUC'] = roc_auc

# Save CV results
results_path = os.path.join(
    LOG_DIR,
    f'training_results_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H.csv'
)
results.to_csv(results_path, index=False)
print(f"\nCV results saved to {results_path}")

# === Final training and evaluation ===
final_cb = FinalMetricsCallback(X_test, y_test)

final_model = DeepScan(window_sizes=WINDOW_SIZES,
                      num_filters=NUM_FILTERS,
                      num_hidden=NUM_HIDDEN)
final_model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='Independent')
final_checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_Independent.keras')
final_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(final_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')
final_early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
final_tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, 'final_model'))

history = final_model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[final_cb, final_tb_cb, final_checkpoint_cb, final_early_stop_cb, save_cb],
    verbose=1
)

# Save final results
results.to_csv(results_path, index=False)
print(f"\nFinal results saved to {results_path}")

best_auc_idx = results[results['Stage'] == 'Independent']['AUC'].idxmax()
best_auc_row = results.loc[best_auc_idx]
print(f"\nBest Independent Test AUC: {best_auc_row['AUC']:.4f} (Epoch {best_auc_row['Epoch']})")



=== Fold 1/5 ===


Model: "DeepScan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_30 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_30             │ (None, 1, 1011, 256)   │       343,296 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_30 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_30 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 607,489 (2.32 MB)

 Trainable params: 607,489 (2.32 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_34 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_34             │ (None, 1, 1011, 256)   │       343,296 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_34 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_34 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_34 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 607,489 (2.32 MB)

 Trainable params: 607,489 (2.32 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.5556 - auc: 0.5635 - loss: 0.7463[Saved model] Epoch 1 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[12]_256F_1024H/CV/fold_5/CV_fold5_epoch01.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 574ms/step - accuracy: 0.5583 - auc: 0.5678 - loss: 0.7439 - val_accuracy: 0.6818 - val_auc: 0.8520 - val_loss: 0.6586
Epoch 2/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.6488 - auc: 0.6942 - loss: 0.6746[Saved model] Epoch 2 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[12]_256F_1024H/CV/fold_5/CV_fold5_epoch02.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 425ms/step - accuracy: 0.6502 - auc: 0.6962 - loss: 0.6731 - val_accuracy: 0.7231 - val_auc: 0.8951 - val_loss: 0.5999
Epoch 3/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.7015 - auc: 0.7927 - loss: 0.6057[Saved model] Epoch 3 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[12]_256F_1024H/CV/fold_5/CV_fold5_epoch03.keras
16

In [ ]:
import os
import time
import math

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.saving import register_keras_serializable

# === Hyperpararmeters ===
BATCH_SIZE      = 128
NUM_CLASSES     = 1
EPOCHS          = 50
NUM_FILTERS     = 256
NUM_HIDDEN      = 1024
WINDOW_SIZES    = [8]
MAX_SEQ_LENGTH  = 1022
EMBEDDING_WIDTH = 1280
LOG_MODEL = os.path.join(LOG_DIR, f'MODELS_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H')
os.makedirs(LOG_MODEL, exist_ok=True)

@register_keras_serializable()
def DeepScan(input_shape=(1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH),
             window_sizes=WINDOW_SIZES,
             num_filters=NUM_FILTERS,
             num_hidden=NUM_HIDDEN,
             num_classes=NUM_CLASSES):
    inputs = tf.keras.Input(shape=input_shape)

    branches = []
    for ws in window_sizes:
        x = layers.SeparableConv2D(
            filters=num_filters,
            kernel_size=(1, ws),
            strides=(1, 1),
            activation='relu',
            padding='valid',
            depthwise_regularizer=tf.keras.regularizers.l2(1e-4),
            pointwise_regularizer=tf.keras.regularizers.l2(1e-4),
            depthwise_initializer='glorot_uniform',
            pointwise_initializer='glorot_uniform'
        )(inputs)

        x = layers.MaxPooling2D(
            pool_size=(1, MAX_SEQ_LENGTH - ws + 1),
            strides=(1, 1),
            padding='valid'
        )(x)

        x = layers.Flatten()(x)
        branches.append(x)

    x = layers.Concatenate()(branches)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(num_hidden, activation='relu', name='fc1')(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='DeepScan')
    return model

# === Callback on CV folds ===
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val, fold):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.fold  = fold
        self.fold_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)
        cm = metrics.confusion_matrix(self.y_val, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()
        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0
        epoch_time = time.time() - self.epoch_start
        results.loc[len(results)] = [
            'CV', self.fold, epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, None,
            None, epoch_time, self.model.count_params()
        ]

class SaveEveryEpochCallback(tf.keras.callbacks.Callback):
    def __init__(self, base_dir, stage='CV', fold=None):
        super().__init__()
        self.base_dir = base_dir
        self.stage = stage
        self.fold = fold
        self.sub_dir = os.path.join(base_dir, stage, f'fold_{fold}' if fold is not None else '')
        os.makedirs(self.sub_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        filename = f"{self.stage}_fold{self.fold}_epoch{epoch+1:02d}.keras" if self.fold is not None else f"{self.stage}_epoch{epoch+1:02d}.keras"
        path = os.path.join(self.sub_dir, filename)
        self.model.save(path)
        print(f"[Saved model] Epoch {epoch+1} saved to {path}")

# === Callback on independent test ===
class FinalMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.epoch_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        y_pred_probs  = self.model.predict(self.X_test, batch_size=BATCH_SIZE, verbose=0).ravel()
        y_pred_labels = (y_pred_probs >= 0.5).astype(int)

        cm = metrics.confusion_matrix(self.y_test, y_pred_labels)
        if cm.size == 1:
            if y_pred_labels[0] == 1:
                TN, FP, FN, TP = 0, 0, cm[0,0], 0
            else:
                TN, FP, FN, TP = cm[0,0], 0, 0, 0
        else:
            TN, FP, FN, TP = cm.ravel()

        Sens = TP/(TP+FN) if TP+FN>0 else 0
        Spec = TN/(TN+FP) if TN+FP>0 else 0
        Acc  = (TP+TN)/(TP+FP+TN+FN) if TP+FP+TN+FN>0 else 0
        denom = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        MCC = (TP*TN - FP*FN)/math.sqrt(denom) if denom>0 else 0
        F1  = 2*TP/(2*TP + FP + FN) if 2*TP+FP+FN>0 else 0

        fpr, tpr, _ = roc_curve(self.y_test, y_pred_probs)
        roc_auc = auc(fpr, tpr)

        epoch_time = time.time() - self.epoch_start_time

        results.loc[len(results)] = [
            'Independent', 'Final', epoch+1, TP, FP, TN, FN,
            Sens, Spec, Acc, MCC, F1, roc_auc,
            None, epoch_time, self.model.count_params()
        ]

# === DataFrame Column ===
results_columns = [
    'Stage', 'Fold', 'Epoch', 'TP', 'FP', 'TN', 'FN',
    'Sens', 'Spec', 'Acc', 'MCC', 'F1', 'AUC',
    'Train_Time', 'Epoch_Time', 'Total_Params'
]
results = pd.DataFrame(columns=results_columns)

# === 5-Fold Cross-Validation + ROC per fold ===
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
    print(f"\n=== Fold {fold}/5 ===")
    X_train_fold, X_val = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val = y_train[train_idx], y_train[val_idx]

    model = DeepScan(window_sizes=WINDOW_SIZES,
                     num_filters=NUM_FILTERS,
                     num_hidden=NUM_HIDDEN)
    model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
    model.summary()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    metrics_cb = MetricsCallback(X_val, y_val, fold)
    tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, f'fold_{fold}'))
    early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='CV', fold=fold)
    checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_CV_{fold}.keras')
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')

    start_fold = time.time()
    model.fit(
        X_train_fold, y_train_fold,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=[metrics_cb, tb_cb, early_stop_cb, save_cb, checkpoint_cb],
        verbose=1,
    )
    train_time = time.time() - start_fold
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'Train_Time'] = train_time

    y_val_probs = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).ravel()
    fpr, tpr, _ = roc_curve(y_val, y_val_probs)
    roc_auc = auc(fpr, tpr)
    results.loc[(results.Stage=='CV') & (results.Fold==fold), 'AUC'] = roc_auc

# Save CV results
results_path = os.path.join(
    LOG_DIR,
    f'training_results_{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H.csv'
)
results.to_csv(results_path, index=False)
print(f"\nCV results saved to {results_path}")

# === Final training and evaluation ===
final_cb = FinalMetricsCallback(X_test, y_test)

final_model = DeepScan(window_sizes=WINDOW_SIZES,
                      num_filters=NUM_FILTERS,
                      num_hidden=NUM_HIDDEN)
final_model.build((None, 1, MAX_SEQ_LENGTH, EMBEDDING_WIDTH))
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

save_cb = SaveEveryEpochCallback(base_dir=LOG_MODEL, stage='Independent')
final_checkpoint_path = os.path.join(LOG_MODEL, f'best_model__{WINDOW_SIZES}_{NUM_FILTERS}F_{NUM_HIDDEN}H_Independent.keras')
final_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(final_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')
final_early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
final_tb_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, 'final_model'))

history = final_model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[final_cb, final_tb_cb, final_checkpoint_cb, final_early_stop_cb, save_cb],
    verbose=1
)

# Save final results
results.to_csv(results_path, index=False)
print(f"\nFinal results saved to {results_path}")

best_auc_idx = results[results['Stage'] == 'Independent']['AUC'].idxmax()
best_auc_row = results.loc[best_auc_idx]
print(f"\nBest Independent Test AUC: {best_auc_row['AUC']:.4f} (Epoch {best_auc_row['Epoch']})")



=== Fold 1/5 ===


Model: "DeepScan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_36 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_36             │ (None, 1, 1015, 256)   │       338,176 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_36 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_36 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 602,369 (2.30 MB)

 Trainable params: 602,369 (2.30 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_40 (InputLayer)     │ (None, 1, 1022, 1280)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_40             │ (None, 1, 1015, 256)   │       338,176 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_40 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_40 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_40 (Concatenate)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 602,369 (2.30 MB)

 Trainable params: 602,369 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.5147 - auc: 0.5181 - loss: 0.7788[Saved model] Epoch 1 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[8]_256F_1024H/CV/fold_5/CV_fold5_epoch01.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 533ms/step - accuracy: 0.5192 - auc: 0.5242 - loss: 0.7745 - val_accuracy: 0.7562 - val_auc: 0.8623 - val_loss: 0.6632
Epoch 2/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - accuracy: 0.6569 - auc: 0.7131 - loss: 0.6669[Saved model] Epoch 2 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[8]_256F_1024H/CV/fold_5/CV_fold5_epoch02.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 379ms/step - accuracy: 0.6593 - auc: 0.7159 - loss: 0.6652 - val_accuracy: 0.8140 - val_auc: 0.8962 - val_loss: 0.6076
Epoch 3/50
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.7016 - auc: 0.7895 - loss: 0.6115[Saved model] Epoch 3 saved to /content/drive/MyDrive/Clathrin-msCNN/log/esm2/MODELS_[8]_256F_1024H/CV/fold_5/CV_fold5_epoch03.keras
16/16 